In [1]:
#상황에 맞게 방법 1 or 2를 선택하여 코드 실행하기

In [30]:
from pydub import AudioSegment
import os
import wave

In [31]:
os.getcwd()

'C:\\Users\\user\\KITA'

# 방법1: wav를 잘라서 5초로 만들되, 3이상 5초 미만짜리는 끝에 padding(무음처리)

In [38]:
input_dir = "C:\\Users\\user\\KITA\\counsel\\" #5초로 자를 파일이 있는 곳
output_dir = "C:\\Users\\user\\KITA\\5sec\\" #5초로 잘린 파일 여기다 저장하기

# Make sure the output directory exists, and if not, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop over all files in the directory
for filename in os.listdir(input_dir):
    if filename.endswith(".wav"):
        # Load an audio file
        audio = AudioSegment.from_wav(input_dir + filename)

        length_audio = len(audio)
        start = 0
        end = 0
        counter = 1

        # Cut audio every 5 seconds
        while start < length_audio:
            end += 5000  # 5000 milliseconds == 5 seconds

            if end > length_audio:  # If the end point is longer than the audio duration
                end = length_audio  # set the end point at the end of the audio

            chunk = audio[start:end]  # extract a piece of the audio

            # If chunk is 3, 4, or 5 seconds long
            if 3000 <= len(chunk) <= 5000:  # Between 3000 and 5000 milliseconds == Between 3 and 5 seconds
                # If chunk is less than 5 seconds, add silence to the end to make it 5 seconds long
                if len(chunk) < 5000:  # Less than 5 seconds
                    silence_duration = 5000 - len(chunk)  # Calculate how much silence we need
                    silence = AudioSegment.silent(duration=silence_duration)  # Generate the necessary silence
                    chunk += silence  # Add the silence to the end of the chunk

                chunk.export(output_dir + f"{os.path.splitext(filename)[0]}_part_{counter:04d}.wav", format="wav")  # save the audio chunk
                counter += 1

            start += 5000  # Move to the next audio chunk

#### 위의 코드에서 패딩이 누락된(5초보다 짧은 파일이 있는지) 것을 확인하고, 누락된 것은 다시 패딩하기

In [40]:
#5초보다 짧은 파일이 있는지 출력

short_files = []

for filename in os.listdir(output_dir):
    if filename.endswith('.wav'):
        with wave.open(os.path.join(output_dir, filename), 'rb') as wav_file:
            frames = wav_file.getnframes()
            frame_rate = wav_file.getframerate()
            duration = frames / float(frame_rate)
            if duration < 5:
                short_files.append(filename)

print(f"Files shorter than 5 seconds: {short_files}")

#다시 패딩 후 초 확인
def add_silence(filename, target_length=5):
    sound = AudioSegment.from_wav(filename)
    silence_needed = (target_length * 1000) - len(sound)  # PyDub works in milliseconds
    print(f"Silence needed for {filename}: {silence_needed} ms")
    silence = AudioSegment.silent(duration=silence_needed)
    padded_sound = sound + silence
    padded_sound.export(filename, format='wav')
    print(f"New duration of {filename}: {len(padded_sound)/1000} s")

for filename in os.listdir(output_dir):
    if filename.endswith('.wav'):
        with wave.open(os.path.join(output_dir, filename), 'rb') as wav_file:
            frames = wav_file.getnframes()
            frame_rate = wav_file.getframerate()
            duration = frames / float(frame_rate)
            print(f"Original duration of {filename}: {duration} s")
            if duration < 5:
                add_silence(os.path.join(output_dir, filename))

Files shorter than 5 seconds: ['counsel_merge_1_part_0004.wav']


# 방법 2: wav를 5초로 자르되, padding 없이 하기

In [ ]:
def process_wav_files(input_dir, output_dir):
    filenames = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.wav')]
    
    combined_audio = AudioSegment.empty()
    
    for file in filenames:
        audio = AudioSegment.from_wav(file)
        
        # If the current audio is exactly 5 seconds
        if len(audio) == 5000:
            print(f"Original 5-second file: {file}")
            save_name = os.path.join(output_dir, os.path.basename(file))
            audio.export(save_name, format="wav")
        
        combined_audio += audio

    # Split combined_audio into 5-second chunks and save
    idx = 0
    while len(combined_audio) >= 5000:
        chunk = combined_audio[:5000]
        chunk.export(os.path.join(output_dir, f"output_{idx}.wav"), format="wav")
        combined_audio = combined_audio[5000:]
        idx += 1

    # Note: Any residual audio less than 5 seconds will be ignored. (마지막 residual 1개가 5초 미만이라면 버려진다)

if __name__ == "__main__":
    input_directory = "D:/user/Downloads/pol 범죄자/"  # replace with your input folder path
    output_directory = "D:/user/Desktop/5sec_full_vp/"  # replace with your output folder path
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        
    process_wav_files(input_directory, output_directory)